In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np

In [3]:
resnet_pretrained = models.resnet101(pretrained=True)
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet101()
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)

In [4]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [5]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    turning_off_keys = [k for k in formatted_keys for l in lst if l in k]
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        if k in turning_off_keys:
            obj.requires_grad = True
        else:
            obj.requires_grad = False

In [6]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [7]:
batch_size = 32

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='../data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [11]:
total_imgs = len(trainset.imgs)

In [12]:
resnet_attn = resnet_attn.cuda()

In [13]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 52672


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

But this is too big a number,
let's try: 
$- (x - 1)^2$ for now

In [14]:
_lambda = 1e-1 #set default

In [15]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval(f'{net}.'+k)
        if name in k:
            res.append(obj)
    return res

In [16]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
#     penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    penality = (1000/n_params)*sum([torch.min(torch.pow(t-2, 2), torch.pow(t, 2)).sum() for t in attns])
    return (_lambda)*(penality)

In [17]:
len(get_params_objs('attn_weights'))

104

In [18]:
compute_attn_loss()

Variable containing:
 198.3133
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [19]:
print_every = 5

In [20]:
def train_k_epoch(k, add_attn=True, score_epoch=False):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    for epoch in range(k):
        running_loss = 0.0
        running_attn_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

            optimizer.zero_grad()
            outputs = resnet_attn(inputs)
            clf_loss = cls_criterion(outputs, labels)
                        
            attn_loss = compute_attn_loss()
            
            if add_attn:
                loss = clf_loss + attn_loss
            else:
                loss = clf_loss

            loss.backward()
            optimizer.step()

            running_loss += clf_loss.data[0]
            running_attn_loss += attn_loss.data[0]

            if i % print_every == 0:
                print('[%5d] iter, [%2f] epoch, classifer loss: %.3f, attn_loss: %.5f ' %
                      (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
                running_loss = 0.0
                running_attn_loss = 0.0
        if score_epoch:
            print(score(batch_size=32))

In [23]:
from tqdm import tqdm
def score(net=resnet_attn, batch_size=batch_size):
    trainset = torchvision.datasets.ImageFolder(root='../data/train', transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    valset = torchvision.datasets.ImageFolder(root='../data/val', transform=transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    train_correct = 0
    val_correct = 0
    
    for inp, label in tqdm(iter(trainloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        train_correct += int(sum(idx.cpu().data == label))
    
    for inp, label in tqdm(iter(valloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        val_correct += int(sum(idx.cpu().data == label))
    
    return {
        'train_accu': train_correct/len(trainset),
        'val_accu': val_correct/len(valset)
    }

Train a fresh fc layer. 
`turn_off_grad_except([])` turns off grads for all weights but the fc layer

In [22]:
turn_off_grad_except(['fc'])
resnet_attn.eval()
train_k_epoch(1,score_epoch=True)

[    1] iter, [0.000000] epoch, classifer loss: 1.006, attn_loss: 39.66265 
[    6] iter, [0.019605] epoch, classifer loss: 4.732, attn_loss: 198.31326 
[   11] iter, [0.039211] epoch, classifer loss: 4.992, attn_loss: 198.31326 
[   16] iter, [0.058816] epoch, classifer loss: 4.583, attn_loss: 198.31326 
[   21] iter, [0.078422] epoch, classifer loss: 4.341, attn_loss: 198.31326 
[   26] iter, [0.098027] epoch, classifer loss: 4.259, attn_loss: 198.31326 
[   31] iter, [0.117633] epoch, classifer loss: 3.973, attn_loss: 198.31326 
[   36] iter, [0.137238] epoch, classifer loss: 3.866, attn_loss: 198.31326 
[   41] iter, [0.156844] epoch, classifer loss: 3.647, attn_loss: 198.31326 
[   46] iter, [0.176449] epoch, classifer loss: 3.862, attn_loss: 198.31326 
[   51] iter, [0.196054] epoch, classifer loss: 3.845, attn_loss: 198.31326 
[   56] iter, [0.215660] epoch, classifer loss: 3.402, attn_loss: 198.31326 
[   61] iter, [0.235265] epoch, classifer loss: 3.574, attn_loss: 198.31326 


FileNotFoundError: [Errno 2] No such file or directory: './data/train'

In [24]:
score(batch_size=32)

100%|██████████| 55/55 [00:24<00:00,  2.28it/s]


{'train_accu': 0.34909937507658373, 'val_accu': 0.30045610034207526}

In [26]:
print_every = 50

In [27]:
turn_off_grad_except(['attn_weights'])
resnet_attn.eval()
train_k_epoch(1,score_epoch=True)

[    1] iter, [0.000000] epoch, classifer loss: 0.048, attn_loss: 3.96627 
[   51] iter, [0.196054] epoch, classifer loss: 2.392, attn_loss: 188.83913 
[  101] iter, [0.392109] epoch, classifer loss: 2.466, attn_loss: 171.21054 
[  151] iter, [0.588163] epoch, classifer loss: 2.553, attn_loss: 154.94150 
[  201] iter, [0.784218] epoch, classifer loss: 2.713, attn_loss: 139.92177 
[  251] iter, [0.980272] epoch, classifer loss: 2.889, attn_loss: 126.10889 


100%|██████████| 55/55 [00:24<00:00,  2.27it/s]

{'train_accu': 0.2956745496875383, 'val_accu': 0.28107183580387685}


In [28]:
turn_off_grad_except(['attn_weights', 'fc'])
resnet_attn.eval()
train_k_epoch(3,score_epoch=True)

[    1] iter, [0.000000] epoch, classifer loss: 0.050, attn_loss: 2.36370 
[   51] iter, [0.196054] epoch, classifer loss: 3.300, attn_loss: 111.04145 
[  101] iter, [0.392109] epoch, classifer loss: 3.359, attn_loss: 97.82188 
[  151] iter, [0.588163] epoch, classifer loss: 3.506, attn_loss: 85.83250 
[  201] iter, [0.784218] epoch, classifer loss: 3.617, attn_loss: 75.01012 
[  251] iter, [0.980272] epoch, classifer loss: 3.654, attn_loss: 65.24837 


100%|██████████| 55/55 [00:24<00:00,  2.27it/s]

{'train_accu': 0.1586815341257199, 'val_accu': 0.15108323831242873}


[    1] iter, [0.000000] epoch, classifer loss: 0.065, attn_loss: 1.19517 
[   51] iter, [0.196054] epoch, classifer loss: 3.682, attn_loss: 55.54878 
[  101] iter, [0.392109] epoch, classifer loss: 3.839, attn_loss: 47.90606 


Process Process-19:
Process Process-20:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    

KeyboardInterrupt: 